<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/NLP_predictText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**NLP**: **(2 methods: (1) fine-tuning LLM and (2) model training from scratch)**

# predict the answer to a question or complete a sentence

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# Read the API key from the file
with open("/content/drive/My Drive/hf_token.txt", "r") as f:
    hf_token = f.read().strip()

from huggingface_hub import login
login(hf_token)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Read the API key from the file
with open('/content/drive/MyDrive/wandb_key.txt', 'r') as f:
    wandb_key = f.read().strip()

# Set the W&B API key
import os
os.environ["WANDB_API_KEY"] = wandb_key

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system

In [7]:
from transformers import Trainer, TrainingArguments
import torch, gc
from datasets import load_dataset, Dataset

torch.cuda.empty_cache()
gc.collect()

0

In [16]:
# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("community-datasets/yahoo_answers_topics")

README.md:   0%|          | 0.00/5.20k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/270M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/21.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [17]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer'],
        num_rows: 1400000
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer'],
        num_rows: 60000
    })
})


In [18]:
print(dataset['train'][0])

{'id': 0, 'topic': 4, 'question_title': "why doesn't an optical mouse work on a glass table?", 'question_content': 'or even on some surfaces?', 'best_answer': 'Optical mice use an LED and a camera to rapidly capture images of the surface beneath the mouse.  The infomation from the camera is analyzed by a DSP (Digital Signal Processor) and used to detect imperfections in the underlying surface and determine motion. Some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the DSP to accurately analyze the surface beneath the mouse.  \\nSince glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  Mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the DSP not to recognize motion properly. When the system is unable to see surface changes associated with movement, the mouse will not work properly.'}


In [19]:
dataset = dataset.remove_columns(["id", "topic"])
print(dataset['train'][0])

{'question_title': "why doesn't an optical mouse work on a glass table?", 'question_content': 'or even on some surfaces?', 'best_answer': 'Optical mice use an LED and a camera to rapidly capture images of the surface beneath the mouse.  The infomation from the camera is analyzed by a DSP (Digital Signal Processor) and used to detect imperfections in the underlying surface and determine motion. Some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the DSP to accurately analyze the surface beneath the mouse.  \\nSince glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  Mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the DSP not to recognize motion properly. When the system is unable to see surface changes associated with movement, the mouse will not work properly.'}


In [ ]:
# Function to merge and tokenize questions
def preprocess_function(examples):
    questions = [
        f"Title: {title} Content: {content}" if title and content else
        title if title else content
        for title, content in zip(examples["question_title"], examples["question_content"])
    ]
    return tokenizer(questions, truncation=True, padding="max_length", max_length=512)

# Apply preprocessing
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["id", "topic", "question_title", "question_content"])

# Set input and target columns
tokenized_dataset = tokenized_dataset.rename_column("best_answer", "labels")

# Verify dataset structure
print(tokenized_dataset)


In [10]:
from transformers import T5Config, T5ForConditionalGeneration, T5TokenizerFast

# Define tokenizer (you can train a new one if needed)
tokenizer = T5TokenizerFast.from_pretrained("t5-small")  # Uses a small tokenizer to save memory

# Preprocessing function
def preprocess_function(examples):
    # Extract English ('en') and Sanskrit ('sn') texts from the 'translation' field in the list
    inputs = tokenizer([example['en'] for example in examples['translation']], padding="max_length", truncation=True, max_length=128)
    labels = tokenizer([example['sn'] for example in examples['translation']], padding="max_length", truncation=True, max_length=128)

    # Add the labels as the 'input_ids' of the Sanskrit tokens
    inputs['labels'] = labels['input_ids']
    return inputs

# Apply preprocessing
train_dataset = dataset["train"].map(preprocess_function, batched=True)
val_dataset = dataset["validation"].map(preprocess_function, batched=True)
test_dataset = dataset["test"].map(preprocess_function, batched=True)



tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/33000 [00:00<?, ? examples/s]

KeyError: 'translation'

In [ ]:
# **Optimized Transformer Config for Low Memory**
config = T5Config(
    vocab_size=32000,  # Reduce if needed
    decoder_start_token_id=tokenizer.pad_token_id,
    d_model=256,        # Smaller embedding size (default is 768)
    num_layers=8,       # Reduce layers (default is 12)
    num_heads=8,        # Reduce attention heads (default is 12)
    d_ff=1024,          # Reduce feed-forward size (default is 3072)
    dropout_rate=0.1
)

# Initialize model
model = T5ForConditionalGeneration(config)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)



In [ ]:
max_index = max(max(seq) for seq in train_dataset['input_ids'])  # Find max token index
print(f"Max token index in dataset: {max_index}, Model vocab size: {model.config.vocab_size}")


In [ ]:
# **Training arguments optimized for memory**
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=30,  # Adjust as per GPU memory
    per_device_eval_batch_size=30,
    gradient_accumulation_steps=2,  # Simulates a larger batch size
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=1000,
    fp16=True,  # Enables mixed precision training
    gradient_checkpointing=True,  # Saves memory
    load_best_model_at_end=True,
    num_train_epochs=2,  # Adjust as needed
    #max_steps=2000,  # Train only for 200 steps instead of completing the full dataset across one epoch or many.
    report_to="none"  # Avoid logging to external services
)

# **Initialize Trainer**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()


In [ ]:
# **Evaluate the Model**
results = trainer.evaluate(test_dataset)
print("Evaluation Results:", results)

# **Save the Fine-Tuned Model**
model.save_pretrained("./english_to_sanskrit_model")
tokenizer.save_pretrained("./english_to_sanskrit_model")



In [ ]:

# Define the preprocessing function
def hi(examples):
    # Tokenize the English sentences (inputs) and return tensors
    inputs = tokenizer(examples['text'], padding="max_length", max_length=64, truncation=True, return_tensors="pt")
    return inputs

# Data: English sentences
data = ["I am very happy today and hungry too.", "Life is good.", "All is well."]

# Wrap it in a dataset
dataset = Dataset.from_dict({"text": data})

# Apply the preprocessing function
dataset = dataset.map(hi, batched=True)

# Get inputs from the dataset
inputs = dataset['input_ids']  # Extract the input ids after preprocessing

# Convert the list of input_ids into a tensor and move to the appropriate device
inputs_tensor = torch.tensor(inputs).to(device)  # Move to device (GPU or CPU)

# Make predictions
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs_tensor,
        max_length=128,
        num_beams=5,
        temperature=0.7,  # Adjust temperature for diversity
        top_p=0.9,  # nucleus sampling
        top_k=50,  # Limit the top-k tokens to sample from
        early_stopping=True
    )

# Decode the predictions (Sanskrit translations)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the translations
for i, text in enumerate(decoded_preds):
    print(f"English: {data[i]}")
    print(f"Sanskrit Translation: {text}")
    print('-' * 50)
